In [1]:
import tkinter
import tkinter.messagebox
import customtkinter
import tkinter.messagebox as messagebox
from tkintermapview import TkinterMapView
from tkinter import filedialog as fd
from pathlib import Path
from PIL import Image

from getlocationtype import *
from getlocationflooddata import *

customtkinter.set_appearance_mode("Dark")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("green")  # Themes: "blue" (standard), "green", "dark-blue"

class App(customtkinter.CTk):
    def __init__(self):
        self.lon = 0
        self.lat = 0

        super().__init__()

        self.iconbitmap("logo.ico")

        # configure window
        self.title("Project T")
        self.geometry(f"{1280}x{800}")

        # configure grid layout (4x4)
        # self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((1, 2), weight=1)
        self.grid_rowconfigure((0, 1), weight=1)


        # create sidebar frame with widgets
        self.sidebar_frame = customtkinter.CTkFrame(self, fg_color=("white", "black"), corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=7, sticky="nsw")
        self.sidebar_frame.grid_rowconfigure(8, weight=1)

        self.app_title = customtkinter.CTkLabel(self.sidebar_frame, text="Project T", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.app_title.grid(row=0, column=0, columnspan=2, padx=20, pady=(20, 10))

        my_image = customtkinter.CTkImage(light_image=Image.open("TITLE.png"),
                            dark_image=Image.open("TITLE.png"),
                            size=(30, 30))
        
        self.app_banner = customtkinter.CTkLabel(self.sidebar_frame, text="", image=my_image)
        self.app_banner.grid(row=1, column=0, columnspan=2, padx=20, pady=(0, 0))


        # titleimage = ImageTk.PhotoImage(file = "title.png")
        # self.title=customtkinter.CTkLabel(self.sidebar_frame,text="")
        # self.title.grid(row=0, sticky="nswe", padx=10, pady=10)
        # self.title_image = CTkImage(titleimage, size=(180,60)) 
        # self.title.configure(image=(self.title_image))

        #create main entry and button
        self.sidebar_button_1 = customtkinter.CTkButton(self.sidebar_frame, text="Load IFC file",command=self.sidebar_button_event, font=customtkinter.CTkFont(size=12, weight="bold"))
        self.sidebar_button_1.grid(row=2, column=0, columnspan=2, padx=20, pady=10)

        # self.entrylon = customtkinter.CTkEntry(self.sidebar_frame, placeholder_text="Lon", width=100)
        # self.entrylon.grid(row=3, column=0, columnspan=1, padx=(20, 0), pady=(20, 20))

        # self.entrylat = customtkinter.CTkEntry(self.sidebar_frame, placeholder_text="Lat", width=100)
        # self.entrylat.grid(row=3, column=1, columnspan=1, padx=(0, 20), pady=(20, 20))

        self.entry = customtkinter.CTkEntry(master=self.sidebar_frame,
                                            placeholder_text="type address", width=150)
        self.entry.grid(row=3, column=0, sticky="we", padx=(30, 0), pady=12)
        self.entry.bind("<Return>", self.search_event)

        self.button_5 = customtkinter.CTkButton(master=self.sidebar_frame, text="Search", width=30, command=self.search_event,font=customtkinter.CTkFont(size=12, weight="bold"))
        self.button_5.grid(row=3, column=1, sticky="we", padx=(30, 30), pady=12)

        self.map_widget = TkinterMapView(self.sidebar_frame, corner_radius=0, height=360)
        self.map_widget.grid(row=4, rowspan=2, column=0, columnspan=2, sticky="nswe", padx=(20, 20), pady=(0, 0))
        self.map_widget.add_right_click_menu_command(label="Select location", command=self.add_marker_event,pass_coords=True)

        # self.sidebar_button_2 = customtkinter.CTkButton(self.sidebar_frame, command=self.sidebar_button_event)
        # self.sidebar_button_2.grid(row=2, column=0, padx=20, pady=10)

        self.sidebar_button_1 = customtkinter.CTkButton(self.sidebar_frame, text="Run Analysis",command=self.sidebar_button_event, font=customtkinter.CTkFont(size=12, weight="bold"))
        self.sidebar_button_1.grid(row=7, column=0, columnspan=2, padx=20, pady=20)

        # self.sidebar_button_3 = customtkinter.CTkButton(self.sidebar_frame, command=self.sidebar_button_event)
        # self.sidebar_button_3.grid(row=3, column=0, padx=20, pady=10)

        self.map_label = customtkinter.CTkLabel(self.sidebar_frame, text="Tile Server:", anchor="w")
        self.map_label.grid(row=8, column=0, padx=(0, 0), pady=(0, 0), sticky = "s")
        self.map_option_menu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["OpenStreetMap", "Google Maps", "Google Satellite", "Stamen BW Map"],
                                                                       command=self.change_map)
        self.map_option_menu.grid(row=8, column=1, padx=(20, 20), pady=(10, 0), sticky = "s")

        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Theme:", anchor="w")
        self.appearance_mode_label.grid(row=9, column=0, padx=0, pady=(0, 10), sticky = "s")
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark"],
                                                                       command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=9, column=1, padx=20, pady=(10, 10), sticky = "s")
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="UI Scaling:", anchor="w")
        self.scaling_label.grid(row=10, column=0, padx=0, pady=(0, 50), sticky = "s")
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
                                                               command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=10, column=1, padx=20, pady=(0, 50), sticky = "s")


        # # create textbox
        # self.textbox = customtkinter.CTkTextbox(self, width=250)
        # self.textbox.grid(row=0, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")

        # create tabview
        # self.main_frame = customtkinter.CTkFrame(self, fg_color=("green", "yellow"), width=600, corner_radius=0)
        # self.main_frame.grid(row=0, column=0, rowspan=4, columnspan=4, sticky="nsew")
        # self.main_frame.grid_rowconfigure(4, weight=0)

        # create radiobutton frame
        # self.radiobutton_frame = customtkinter.CTkFrame(self)
        # self.radiobutton_frame.grid(row=0, column=3, padx=(20, 20), pady=(20, 0), sticky="nsew")
        # self.radio_var = tkinter.IntVar(value=0)
        # self.label_radio_group = customtkinter.CTkLabel(master=self.radiobutton_frame, text="CTkRadioButton Group:")
        # self.label_radio_group.grid(row=0, column=2, columnspan=1, padx=10, pady=10, sticky="")
        # self.radio_button_1 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=0)
        # self.radio_button_1.grid(row=1, column=2, pady=10, padx=20, sticky="n")
        # self.radio_button_2 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=1)
        # self.radio_button_2.grid(row=2, column=2, pady=10, padx=20, sticky="n")
        # self.radio_button_3 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=2)
        # self.radio_button_3.grid(row=3, column=2, pady=10, padx=20, sticky="n")

        # create slider and progressbar frame
        # self.slider_progressbar_frame = customtkinter.CTkFrame(self, fg_color="transparent")
        # self.slider_progressbar_frame.grid(row=1, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")
        # self.slider_progressbar_frame.grid_columnconfigure(0, weight=1)
        # self.slider_progressbar_frame.grid_rowconfigure(4, weight=1)
        # self.seg_button_1 = customtkinter.CTkSegmentedButton(self.slider_progressbar_frame)
        # self.seg_button_1.grid(row=0, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        # self.progressbar_1 = customtkinter.CTkProgressBar(self.slider_progressbar_frame)
        # self.progressbar_1.grid(row=1, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        # self.progressbar_2 = customtkinter.CTkProgressBar(self.slider_progressbar_frame)
        # self.progressbar_2.grid(row=2, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        # self.slider_1 = customtkinter.CTkSlider(self.slider_progressbar_frame, from_=0, to=1, number_of_steps=4)
        # self.slider_1.grid(row=3, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        # self.slider_2 = customtkinter.CTkSlider(self.slider_progressbar_frame, orientation="vertical")
        # self.slider_2.grid(row=0, column=1, rowspan=5, padx=(10, 10), pady=(10, 10), sticky="ns")
        # self.progressbar_3 = customtkinter.CTkProgressBar(self.slider_progressbar_frame, orientation="vertical")
        # self.progressbar_3.grid(row=0, column=2, rowspan=5, padx=(10, 20), pady=(10, 10), sticky="ns")

        # create scrollable frame
        # self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="CTkScrollableFrame")
        # self.scrollable_frame.grid(row=1, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        # self.scrollable_frame.grid_columnconfigure(0, weight=1)

        self.scrollable_frame1 = customtkinter.CTkScrollableFrame(self, fg_color=("white", "black"), label_text="IFC Data & Enviromental Data", label_fg_color=("white", "black"), scrollbar_button_color=("white", "black"), scrollbar_button_hover_color=("white", "black"))
        self.scrollable_frame1.grid(row=0, column=1, padx=(20, 20), pady=(20, 0), sticky="nsew")
        self.scrollable_frame1.grid_columnconfigure(0, weight=1)

        self.title=customtkinter.CTkLabel(self.scrollable_frame1,text="xxxxxxxxxxxx")
        self.title.grid(row=0, sticky="nswe", padx=10, pady=10)

        self.scrollable_frame2 = customtkinter.CTkScrollableFrame(self, fg_color=("white", "black"), label_text="Results Graph", label_fg_color=("white", "black"), scrollbar_button_color=("white", "black"), scrollbar_button_hover_color=("white", "black"))
        self.scrollable_frame2.grid(row=0, column=2, padx=(0, 20), pady=(20, 0), sticky="nsew")
        self.scrollable_frame2.grid_columnconfigure(0, weight=1)

        self.tabview = customtkinter.CTkTabview(self.scrollable_frame2, fg_color=("white", "black"))
        self.tabview.grid(row=0, column=0, padx=(10, 10), pady=(0, 0), sticky="nsew")
        self.tabview.add("Very Low Chance")
        self.tabview.add("Low Chance")
        self.tabview.add("Medium Chance")
        self.tabview.add("High Chance")
        self.tabview.tab("Very Low Chance").grid_columnconfigure(0, weight=1)  # configure grid of individual tabs
        self.tabview.tab("High Chance").grid_columnconfigure(0, weight=1)

        self.optionmenu_1 = customtkinter.CTkOptionMenu(self.tabview.tab("High Chance"), dynamic_resizing=False,
                                                        values=["Value 1", "Value 2", "Value Long Long Long"])
        self.optionmenu_1.grid(row=0, column=0, padx=20, pady=(20, 10))
        self.combobox_1 = customtkinter.CTkComboBox(self.tabview.tab("High Chance"),
                                                    values=["Value 1", "Value 2", "Value Long....."])
        self.combobox_1.grid(row=1, column=0, padx=20, pady=(10, 10))
        self.string_input_button = customtkinter.CTkButton(self.tabview.tab("High Chance"), text="Open CTkInputDialog",
                                                           command=self.open_input_dialog_event)
        self.string_input_button.grid(row=2, column=0, padx=20, pady=(10, 10))
        self.label_tab_2 = customtkinter.CTkLabel(self.tabview.tab("Very Low Chance"), text="CTkLabel on Tab 2")
        self.label_tab_2.grid(row=0, column=0, padx=20, pady=20)

        self.scrollable_frame3 = customtkinter.CTkScrollableFrame(self, fg_color=("white", "black"), label_text="Flood Analysis", label_fg_color=("white", "black"), scrollbar_button_color=("white", "black"), scrollbar_button_hover_color=("white", "black"))
        self.scrollable_frame3.grid(row=1, column=1, padx=(20, 20), pady=(20, 0), sticky="nsew")
        self.scrollable_frame3.grid_columnconfigure(0, weight=1)

        self.title=customtkinter.CTkLabel(self.scrollable_frame3,text="xxxxxxxxxxxx")
        self.title.grid(row=0, sticky="nswe", padx=10, pady=10)

        self.scrollable_frame4 = customtkinter.CTkScrollableFrame(self, fg_color=("white", "black"), label_text="Temperature Analysis", label_fg_color=("white", "black"), scrollbar_button_color=("white", "black"), scrollbar_button_hover_color=("white", "black"))
        self.scrollable_frame4.grid(row=1, column=2, padx=(0, 20), pady=(20, 0), sticky="nsew")
        self.scrollable_frame4.grid_columnconfigure(0, weight=1)

        self.title=customtkinter.CTkLabel(self.scrollable_frame4,text="xxxxxxxxxxxx")
        self.title.grid(row=0, sticky="nswe", padx=10, pady=10)

        self.main_button_1 = customtkinter.CTkButton(master=self, fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), text="Export", width=200)
        self.main_button_1.grid(row=3, column=2, pady=(20, 20), padx=(30, 30), sticky="e")


        # self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="Results")
        # self.scrollable_frame.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        # self.scrollable_frame.grid_columnconfigure(0, weight=1)

        # self.scrollable_frame_switches = []
        # # for i in range(100):
        # #     switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"CTkSwitch {i}")
        # #     switch.grid(row=i, column=0, padx=10, pady=(0, 20))
        # switch = customtkinter.CTkTextbox(self,  fg_color=("red", "black"), scrollbar_button_color=("white", "black"), scrollbar_button_hover_color =("white", "black"), width=250)
        # switch.grid(row=1, column=2, padx=30, pady=(50, 30), sticky="nsew")
        # switch.insert("0.0", "Some example text!\n" * 50)
        # self.scrollable_frame_switches.append(switch)

        # # create checkbox and switch frame
        # self.checkbox_slider_frame = customtkinter.CTkFrame(self)
        # self.checkbox_slider_frame.grid(row=1, column=3, padx=(20, 20), pady=(20, 0), sticky="nsew")
        # self.checkbox_1 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        # self.checkbox_1.grid(row=1, column=0, pady=(20, 0), padx=20, sticky="n")
        # self.checkbox_2 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        # self.checkbox_2.grid(row=2, column=0, pady=(20, 0), padx=20, sticky="n")
        # self.checkbox_3 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        # self.checkbox_3.grid(row=3, column=0, pady=20, padx=20, sticky="n")

        # set default values
        # self.sidebar_button_3.configure(state="disabled", text="Disabled CTkButton")
        # self.checkbox_3.configure(state="disabled")
        # self.checkbox_1.select()
        # self.scrollable_frame_switches[0].select()
        # self.scrollable_frame_switches[4].select()
        # self.radio_button_3.configure(state="disabled")
        self.appearance_mode_optionemenu.set("Dark")
        self.scaling_optionemenu.set("100%")
        self.optionmenu_1.set("CTkOptionmenu")
        self.combobox_1.set("CTkComboBox")
        # self.slider_1.configure(command=self.progressbar_2.set)
        # self.slider_2.configure(command=self.progressbar_3.set)
        # self.progressbar_1.configure(mode="indeterminnate")
        # self.progressbar_1.start()
        # self.textbox.insert("0.0", "CTkTextbox\n\n" + "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua.\n\n" * 20)
        # self.seg_button_1.configure(values=["CTkSegmentedButton", "Value 2", "Value 3"])
        # self.seg_button_1.set("Value 2")

    def open_input_dialog_event(self):
        dialog = customtkinter.CTkInputDialog(text="Type in a number:", title="CTkInputDialog")
        print("CTkInputDialog:", dialog.get_input())

    def change_appearance_mode_event(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    def sidebar_button_event(self):
        print("sidebar_button click")
        global filename

        filetypes = (('IFC files', '*.ifc'),('All files', '*.*'))

        filename = fd.askopenfilename(title='Open a file', initialdir="C:\\", filetypes=filetypes)
        filename = Path(filename).stem

        # Check if file extension is ".ifc"
        if Path(filename).suffix != ".ifc":
            messagebox.showerror("Error", "Invalid file type. Only IFC files are allowed.")
            return
        
        loadedifc = filename + ".ifc"
        print(loadedifc)
    
    def search_event(self, event=None):
        self.map_widget.set_address(self.entry.get())

    def change_map(self, new_map: str):
        if new_map == "OpenStreetMap":
            self.map_widget.set_tile_server("https://a.tile.openstreetmap.org/{z}/{x}/{y}.png")
        elif new_map == "Google Maps":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=m&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)
        elif new_map == "Google Satellite":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)
        elif new_map == "Stamen BW Map":
            self.map_widget.set_tile_server("http://a.tile.stamen.com/toner/{z}/{x}/{y}.png")

    def add_marker_event(self, coords):
        self.map_widget.delete_all_marker()
        self.map_widget.delete_all_path()
        # self.map_widget.set_address(self.Location_adres.get(), marker=True)
        global lon
        lon = coords[0]
        global lat
        lat = coords[1]
        # # print(coords)
        # # print(self.lat, self.lon)
        # locationtype = gettypelocation(self.lon, self.lat)
        # print(coords)
        # print(self.lat, self.lon)
        locationtype = gettypelocation(lon, lat)
        print(locationtype, lon, lat)

    def runsimulation(locationtype):
        getflooddata(lon, lat)
        return

if __name__ == "__main__":
    app = App()
    app.mainloop()

In [ ]:
# https://github.com/TomSchimansky/TkinterMapView for map